In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
df = pd.read_csv("hacktrain.csv")
test_data = pd.read_csv("hacktest.csv")

In [20]:
df.drop(columns=['Unnamed: 0', 'ID'], inplace=True)
test_data.drop(columns=['Unnamed: 0'], inplace=True)

In [21]:
df.isnull().sum()

,0
class,0
20150720_N,560
20150602_N,1200
20150517_N,800
20150501_N,960
20150415_N,480
20150330_N,1120
20150314_N,720
20150226_N,1360
20150210_N,640


In [22]:
df.fillna(df.mean(numeric_only=True), inplace=True)
df.isnull().sum()

,0
class,0
20150720_N,0
20150602_N,0
20150517_N,0
20150501_N,0
20150415_N,0
20150330_N,0
20150314_N,0
20150226_N,0
20150210_N,0


In [23]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split into features and target
X = df.drop(columns=['class'])
y = df['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
X_scaled = scaler.fit_transform(X)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)
skf = StratifiedKFold(n_splits=5, shuffle=True)

classification_reports = []

# Perform cross-validation
for train_index, test_index in skf.split(X_resampled, y_resampled):
    X_train, X_test = X_resampled[train_index], X_resampled[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]

    # Initialize and train a stronger model (RandomForestClassifier)
    model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)

    # Make predictions on the test set of the current fold
    y_pred_validation = model.predict(X_test)

    report = classification_report(
        y_test,
        y_pred_validation,
        labels=list(range(len(label_encoder.classes_))),
        target_names=label_encoder.classes_
    )
    classification_reports.append(report)

# Classification report with all original class labels
print("Classification Report on Validation Data:")
for i, report in enumerate(classification_reports):
    print(f"\nFold {i+1}:\n{report}")

Classification Report on Validation Data:

Fold 1:
              precision    recall  f1-score   support

        farm       0.99      0.99      0.99      1232
      forest       1.00      0.99      0.99      1232
       grass       1.00      1.00      1.00      1231
  impervious       0.99      0.99      0.99      1232
     orchard       1.00      1.00      1.00      1232
       water       1.00      1.00      1.00      1232

    accuracy                           1.00      7391
   macro avg       1.00      1.00      1.00      7391
weighted avg       1.00      1.00      1.00      7391


Fold 2:
              precision    recall  f1-score   support

        farm       0.98      0.99      0.99      1231
      forest       1.00      0.98      0.99      1232
       grass       1.00      1.00      1.00      1232
  impervious       0.99      0.99      0.99      1232
     orchard       1.00      1.00      1.00      1232
       water       1.00      1.00      1.00      1232

    accuracy     

In [24]:
final_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
final_model.fit(X_resampled, y_resampled)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [25]:
test_data = pd.read_csv("hacktest.csv")
test_data.shape

(2845, 29)

In [26]:
ID=test_data['ID']
test_data.drop(['ID', 'Unnamed: 0'], axis=1, inplace=True)

In [27]:
test_data.fillna(df.mean(numeric_only=True), inplace=True)

In [28]:
test_data_scaled = scaler.transform(test_data)

In [29]:
y_test_pred = final_model.predict(test_data_scaled)

In [30]:
y_decoded = label_encoder.inverse_transform(y_test_pred)
y_decoded

array(['farm', 'orchard', 'forest', ..., 'water', 'water', 'water'],
      dtype=object)

In [31]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})

In [32]:
result

,ID,class
0,1,farm
1,2,orchard
2,3,forest
3,4,orchard
4,5,orchard
...,...,...
2840,2841,water
2841,2842,water
2842,2843,water
2843,2844,water


from matplotlib import pyplot as plt
result['ID'].plot(kind='hist', bins=20, title='ID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
result.groupby('class').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['ID']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'ID'}, axis=1)
              .sort_values('ID', ascending=True))
  xs = counted['ID']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = result.sort_values('ID', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('class')):
  _plot_series(series, series_name, i)
  fig.legend(title='class', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ID')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
result['ID'].plot(kind='line', figsize=(8, 4), title='ID')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(result['class'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(result, x='ID', y='class', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [33]:
result.to_csv("submission.csv", index=False) #this file will appear under the output section of the right navbar. You need to submit this csv file

In [34]:
print("✅ submission.csv generated successfully.")


✅ submission.csv generated successfully.
